# Learning Curve for Multi-Spike Tempotron

## 1. Run the function-containing cells below

In [1]:
import numpy as np
from main_gen import gen_features

def gen_background_data(n, fr, duration): 
    dt = 0.001 #bin (s)
    gen_bg = np.random.random((n, np.rint(duration/dt).astype(int)))<fr*dt
    gen_bg = gen_bg.astype(int)
    return gen_bg

def kernel_fn(length, tau_mem, tau_syn, time_ij):
    time = np.arange(0., length, 1.) #ms
    kernel = np.zeros(length)
    eta = tau_mem/tau_syn
    V_norm = eta**(eta/(eta-1))/(eta-1)
    for count in range(length):
        kernel[count] = V_norm*(np.exp(-(time[count]-time_ij)/tau_mem)-np.exp(-(time[count]-time_ij)/tau_syn))
    return kernel

In [4]:
### From voltage calculation code by Alex ###

def get_memory_len(kernel_array, ratio):
    arr = (kernel_array - ratio*kernel_array.max())[::-1]
    memory_len = len(kernel_array) - np.searchsorted(arr, 0)
    return memory_len

def presyn_input(data):
    datalen = data.shape[1]
    presyn_input = np.zeros(data.shape)
    for neuron, ith_bin in zip(*np.where(data)):
        mem_len = min(syn_memory_len, datalen - ith_bin)
        presyn_input[neuron,ith_bin:ith_bin+mem_len] += syn_kernel[:mem_len]
    return presyn_input

## 2. Preset Parameters

Set the required parameters, then run the cell below to generate features (in dictionary form), postsynaptic potential (syn_kernel), and spike-triggered reset (ref_kernel). 

In [2]:
n = 500
n_fea = 10 #number of different features
fr = 5 #average firing rate of each input neuron (Hz)
T_fea = 0.05 #T_fea: feature duration (s)
dt = 0.001 #bin: discrete time in second
tau_mem = 20
tau_syn = 5
time_ij = 0
init_kernel_len = 200

np.random.seed(1000000000)
features = gen_features(n_fea, n, fr, dt, T_fea)

syn_kernel = kernel_fn(init_kernel_len, tau_mem, tau_syn, time_ij)
syn_memory_len = get_memory_len(syn_kernel, ratio=0.001)
syn_kernel = syn_kernel[:syn_memory_len]

ref_kernel = np.exp(- np.arange(1000) / tau_mem)
ref_memory_len = get_memory_len(ref_kernel, ratio=0.001)
ref_kernel = ref_kernel[:ref_memory_len]

## 3. Generate Unweighted Inputs

Run the cell below to generate unweighted inputs for n probe trials.

DO NOT run this cell if you already have the file containing generated probe trials (learning_curve_inputs.npy)!

In [ ]:
n_probe_trial = 100
learning_curve_inputs = []

for i in range(n_probe_trial): 
    np.random.seed(i*100000)
    bg_data = gen_background_data(n, fr, 1.95)
    data_null = np.insert(bg_data, 975, np.zeros((50, n)), axis = 1)
    syn_input_null = presyn_input(data_null)
    learning_curve_inputs.append(syn_input_null)

np.save("learning_curve_inputs", learning_curve_inputs)

Run the cell below to generate unweighted inputs for features.

DO NOT run this cell if you already have the 'feature_inputs.npy' file!

In [4]:
feature_inputs = []

for feature, spikes in features.items():
    new_spikes = np.append(spikes, np.zeros((n, 150)), axis = 1)
    fea_input = presyn_input(new_spikes)
    feature_inputs.append(fea_input)

np.save("feature_inputs", feature_inputs)

## 4. Computing Neural Responses

Run the function-containing cells below. Uncomment the 'np.savez()' lines in neural_response and noisy_neural_response functions if the input data is large to avoid losing of computed results when encountering problems.

##### Functions for computing neural responses WITHOUT noise

In [7]:
def add_fea(n_inputs, fea_input, T_fea):
    inputs = np.copy(n_inputs)
    start = int((n_inputs.shape[2] - T_fea*1000)/2)
    for n in inputs:
        n[:, start:start+fea_input.shape[1]] += fea_input
    return inputs

In [8]:
def neural_response(omega_list, n_unweighted_inputs, fea_inputs, T_fea, theta):
    R_fea_list = []
    tot_input = np.zeros((1000, 100, 2000))
    count = 0
    for cycle in omega_list:
        tot_input[count] = (cycle[np.newaxis, :, np.newaxis] * n_unweighted_inputs[:, :, :]).sum(axis = 1)
        count += 1
    null_spike_count = np.zeros(tot_input.shape[:-1])
    for ith_bin in range(tot_input.shape[-1]):
        null_spike_mask = tot_input[:,:,ith_bin] >= theta
        null_spike_count += null_spike_mask
        mem_len = min(ref_memory_len, tot_input.shape[-1] - ith_bin)
        tot_input[:,:,ith_bin:ith_bin+mem_len] -= null_spike_mask[:,:,np.newaxis] * ref_kernel[:mem_len]
    R_null_list = np.mean(null_spike_count, axis = 1)/(n_unweighted_inputs.shape[2]/1000)
    #np.savez("R_null_list", R_null_list, null_spike_count)
    for fea_input in fea_inputs:
        fea_unweighted_inputs = add_fea(n_unweighted_inputs, fea_input, T_fea)
        fea_tot_input = np.zeros((1000, 100, 2000))
        count = 0
        for cycle in omega_list:
            fea_tot_input[count] = (cycle[np.newaxis, :, np.newaxis] * fea_unweighted_inputs[:, :, :]).sum(axis = 1)
            count += 1
        spike_count = np.zeros(fea_tot_input.shape[:-1])
        for ith_bin in range(fea_tot_input.shape[-1]):
            spike_mask = fea_tot_input[:,:,ith_bin] >= theta
            spike_count += spike_mask
            mem_len = min(ref_memory_len, fea_tot_input.shape[-1] - ith_bin)
            fea_tot_input[:,:,ith_bin:ith_bin+mem_len] -= spike_mask[:,:,np.newaxis] * ref_kernel[:mem_len]
        fea_spike_count = spike_count - null_spike_count
        R_fea_list.append(np.mean(fea_spike_count, axis = 1))
        #np.savez("R_fea_list", R_fea_list)
    return R_fea_list, R_null_list

##### Functions for computing neural responses WITH noise

In [ ]:
# randomly delete the spikes
# delete spikes --> noise = 0
# add spikes --> noise = 1

def add_noise(raw_feature, probability, noise):
    noisy_fea = np.copy(raw_feature)
    spike = 0 if noise else 1
    data = list(zip(*np.where(noisy_fea)))
    test_data = list(zip(*np.where(noisy_fea == spike)))
    for i, j in test_data:
        if np.random.random()*len(test_data) <= probability*len(data):
            noisy_fea[i, j] = noise
    return noisy_fea

In [ ]:
# spike time jittering --> jitter within the range of feature duration: range = 0, 50 (bin)

def add_jitter(raw_feature, mean, st_dev):
    raw_fea = np.copy(raw_feature)
    noisy_fea = np.zeros(raw_fea.shape)
    time = np.where(raw_fea)[1]
    jitter = [int(round(i)) for i in list(np.random.normal(mean, st_dev, time.shape[0]))]
    new_time = time + jitter
    count = 0
    for t in new_time:
        if t < 0:
            new_time[count] = 0
        elif t >= raw_fea.shape[1]:
            new_time[count] = raw_fea.shape[1] - 1
        count += 1
    noisy_fea[(np.where(raw_fea)[0], new_time)] = 1
    return noisy_fea

In [ ]:
def add_noisy_fea(n_inputs, raw_fea, T_fea, probability, noise, jitter_mean, jitter_SD):
    inputs = np.copy(n_inputs)
    start = int((inputs.shape[2] - T_fea*1000)/2)
    for n_input in inputs:
        noisy_fea = add_noise(raw_fea, probability, noise)
        if jitter_SD:
            noisy_fea = add_jitter(noisy_fea, jitter_mean, jitter_SD)
        new_fea = np.append(noisy_fea, np.zeros((n, 150)), axis = 1)
        fea_input = presyn_input(new_fea)
        n_input[:, start:start+fea_input.shape[1]] += fea_input
    return inputs

In [ ]:
def noisy_neural_response(omega_list, n_unweighted_inputs, raw_feas, T_fea, theta, probability, noise, jitter_mean, jitter_SD):
    R_fea_list = []
    tot_input = np.zeros((1000, 100, 2000))
    count = 0
    for cycle in omega_list:
        tot_input[count] = (cycle[np.newaxis, :, np.newaxis] * n_unweighted_inputs[:, :, :]).sum(axis = 1)
        count += 1
    null_spike_count = np.zeros(tot_input.shape[:-1])
    for ith_bin in range(tot_input.shape[-1]):
        null_spike_mask = tot_input[:,:,ith_bin] >= theta
        null_spike_count += null_spike_mask
        mem_len = min(ref_memory_len, tot_input.shape[-1] - ith_bin)
        tot_input[:,:,ith_bin:ith_bin+mem_len] -= null_spike_mask[:,:,np.newaxis] * ref_kernel[:mem_len]
    R_null_list = np.mean(null_spike_count, axis = 1)/(n_unweighted_inputs.shape[2]/1000)
    #np.savez("R_null_list", R_null_list, null_spike_count)
    for feature, raw_fea in raw_feas.items():
        fea_unweighted_inputs = add_noisy_fea(n_unweighted_inputs, raw_fea, T_fea, probability, noise, jitter_mean, jitter_SD) 
        fea_tot_input = np.zeros((1000, 100, 2000))
        count = 0
        for cycle in omega_list:
            fea_tot_input[count] = (cycle[np.newaxis, :, np.newaxis] * fea_unweighted_inputs[:, :, :]).sum(axis = 1)
            count += 1
        spike_count = np.zeros(fea_tot_input.shape[:-1])
        for ith_bin in range(fea_tot_input.shape[-1]):
            spike_mask = fea_tot_input[:,:,ith_bin] >= theta
            spike_count += spike_mask
            mem_len = min(ref_memory_len, fea_tot_input.shape[-1] - ith_bin)
            fea_tot_input[:,:,ith_bin:ith_bin+mem_len] -= spike_mask[:,:,np.newaxis] * ref_kernel[:mem_len]
        fea_spike_count = spike_count - null_spike_count
        R_fea_list.append(np.mean(fea_spike_count, axis = 1))
        #np.savez("R_fea_list", R_fea_list)
    return R_fea_list, R_null_list

## Run the cells below to compute the neural responses.

Run the cells below to compute the neural responses, which will be used to plot learning curves.

Use 'theta_omegaXFXS_list.npy' for computing neural responses of updated synaptic efficacies obtained from multi-spike tempotron training (computed from gradient of theta critical). To compute the neural responses of synaptic efficacies obtained from correlation-based training, use 'corr_omegaXFXS_list.npy' by commenting the first line and uncommenting the second line of the cells.

Also, remember to comment and uncommment the corresponding 'np.savez()' lines.

### 4.1 Neural Responses WITHOUT Noise

#### 4.1.1 Identification of one feature with one spike

In [3]:
omega = np.load("theta_omega1F1S_list.npy") # updated omega list after training
#omega = np.load("corr_omega1F1S_list.npy")
l_curve_inputs = np.load("learning_curve_inputs.npy") # unweighted inputs for a list of probe trials (n = 100)
fea_inputs = np.load("feature_inputs.npy") # unweighted inputs of features
theta = 1 # threshold

R_fea_list, R_null_list = neural_response(omega, l_curve_inputs, fea_inputs, T_fea, theta)
np.savez("theta_neural_responses_1F1S", R_fea_list, R_null_list)
#np.savez("corr_neural_responses_1F1S", R_fea_list, R_null_list)

#### 4.1.2 Identification of one feature with multiple spikes

In [3]:
omega = np.load("theta_omega1F5S_list.npy") # updated omega list after training
#omega = np.load("corr_omega1F5S_list.npy")
l_curve_inputs = np.load("learning_curve_inputs.npy") # unweighted inputs for a list of probe trials (n = 100)
fea_inputs = np.load("feature_inputs.npy") # unweighted inputs of features
theta = 1 # threshold

R_fea_list, R_null_list = neural_response(omega, l_curve_inputs, fea_inputs, T_fea, theta)
np.savez("theta_neural_responses_1F5S", R_fea_list, R_null_list)
#np.savez("corr_neural_responses_1F5S", R_fea_list, R_null_list)

#### 4.1.3 Identification of multiple features with one spike per feature

In [3]:
omega = np.load("theta_omega5F1S_list.npy") # updated omega list after training
#omega = np.load("corr_omega5F1S_list.npy")
l_curve_inputs = np.load("learning_curve_inputs.npy") # unweighted inputs for a list of probe trials (n = 100)
fea_inputs = np.load("feature_inputs.npy") # unweighted inputs of features
theta = 1 # threshold

R_fea_list, R_null_list = neural_response(omega, l_curve_inputs, fea_inputs, T_fea, theta)
np.savez("theta_neural_responses_5F1S", R_fea_list, R_null_list)
#np.savez("corr_neural_responses_5F1S", R_fea_list, R_null_list)

#### 4.1.4 Identification of multiple features with different number of spikes for different features

In [3]:
omega = np.load("theta_omega5FmS_list.npy") # updated omega list after training
#omega = np.load("corr_omega5FmS_list.npy")
l_curve_inputs = np.load("learning_curve_inputs.npy") # unweighted inputs for a list of probe trials (n = 100)
fea_inputs = np.load("feature_inputs.npy") # unweighted inputs of features
theta = 1 # threshold

R_fea_list, R_null_list = neural_response(omega, l_curve_inputs, fea_inputs, T_fea, theta)
np.savez("theta_neural_responses_5FmS", R_fea_list, R_null_list)
#np.savez("corr_neural_responses_5FmS", R_fea_list, R_null_list)

#### 4.1.5 Identification of multiple features with a fixed number of spikes (>1) for each feature

In [3]:
omega = np.load("theta_omega5F5S_list.npy") # updated omega list after training
#omega = np.load("corr_omega5F5S_list.npy")
l_curve_inputs = np.load("learning_curve_inputs.npy") # unweighted inputs for a list of probe trials (n = 100)
fea_inputs = np.load("feature_inputs.npy") # unweighted inputs of features
theta = 1 # threshold

R_fea_list, R_null_list = neural_response(omega, l_curve_inputs, fea_inputs, T_fea, theta)
np.savez("theta_neural_responses_5F5S", R_fea_list, R_null_list)
#np.savez("corr_neural_responses_5F5S", R_fea_list, R_null_list)

### 4.2 Neural Responses WITH Noise

#### 4.2.1 Identification of one feature with one spike

In [ ]:
#omega = np.load("theta_omega1F1S_list.npy") # updated omega list after training
omega = np.load("corr_omega1F1S_list.npy")
l_curve_inputs = np.load("learning_curve_inputs.npy") # unweighted inputs for a list of probe trials (n = 10)
raw_fea = features # raw data of features (in dictionary form)
theta = 1 # threshold
probability = 0.2 # range = (0, 1)
noise = 0 # delete spikes --> 0, add spikes --> 1
jitter_mean = 0 # Gaussian distribution
jitter_SD = 2 # Gaussian distribution (in number of bins)

R_fea_list, R_null_list = noisy_neural_response(omega, l_curve_inputs, raw_fea, T_fea, theta, probability, noise, jitter_mean, jitter_SD) 
#np.savez("noisy_theta_neural_responses_1F1S", R_fea_list, R_null_list)
np.savez("noisy_corr_neural_responses_1F1S", R_fea_list, R_null_list)


#### 4.2.2 Identification of one feature with multiple spikes

In [ ]:
#omega = np.load("theta_omega1F5S_list.npy") # updated omega list after training
omega = np.load("corr_omega1F5S_list.npy")
l_curve_inputs = np.load("learning_curve_inputs.npy") # unweighted inputs for a list of probe trials (n = 10)
raw_fea = features # raw data of features (in dictionary form)
theta = 1 # threshold
probability = 0.2 # range = (0, 1)
noise = 0 # delete spikes --> 0, add spikes --> 1
jitter_mean = 0 # Gaussian distribution
jitter_SD = 2 # Gaussian distribution (in number of bins)

R_fea_list, R_null_list = noisy_neural_response(omega, l_curve_inputs, raw_fea, T_fea, theta, probability, noise, jitter_mean, jitter_SD) 
#np.savez("noisy_theta_neural_responses_1F5S", R_fea_list, R_null_list)
np.savez("noisy_corr_neural_responses_1F5S", R_fea_list, R_null_list)


#### 4.2.3 Identification of multiple features with one spike per feature

In [ ]:
#omega = np.load("theta_omega5F1S_list.npy") # updated omega list after training
omega = np.load("corr_omega5F1S_list.npy")
l_curve_inputs = np.load("learning_curve_inputs.npy") # unweighted inputs for a list of probe trials (n = 10)
raw_fea = features # raw data of features (in dictionary form)
theta = 1 # threshold
probability = 0.2 # range = (0, 1)
noise = 0 # delete spikes --> 0, add spikes --> 1
jitter_mean = 0 # Gaussian distribution
jitter_SD = 2 # Gaussian distribution (in number of bins)

R_fea_list, R_null_list = noisy_neural_response(omega, l_curve_inputs, raw_fea, T_fea, theta, probability, noise, jitter_mean, jitter_SD) 
#np.savez("noisy_theta_neural_responses_5F1S", R_fea_list, R_null_list)
np.savez("noisy_corr_neural_responses_5F1S", R_fea_list, R_null_list)


#### 4.2.4 Identification of multiple features with different number of spikes for different features

In [ ]:
#omega = np.load("theta_omega5FmS_list.npy") # updated omega list after training
omega = np.load("corr_omega5FmS_list.npy")
l_curve_inputs = np.load("learning_curve_inputs.npy") # unweighted inputs for a list of probe trials (n = 10)
raw_fea = features # raw data of features (in dictionary form)
theta = 1 # threshold
probability = 0.2 # range = (0, 1)
noise = 0 # delete spikes --> 0, add spikes --> 1
jitter_mean = 0 # Gaussian distribution
jitter_SD = 2 # Gaussian distribution (in number of bins)

R_fea_list, R_null_list = noisy_neural_response(omega, l_curve_inputs, raw_fea, T_fea, theta, probability, noise, jitter_mean, jitter_SD) 
#np.savez("noisy_theta_neural_responses_5FmS", R_fea_list, R_null_list)
np.savez("noisy_corr_neural_responses_5FmS", R_fea_list, R_null_list)


#### 4.2.5 Identification of multiple features with a fixed number of spikes (>1) for each feature

In [ ]:
#omega = np.load("theta_omega5F5S_list.npy") # updated omega list after training
omega = np.load("corr_omega5F5S_list.npy")
l_curve_inputs = np.load("learning_curve_inputs.npy") # unweighted inputs for a list of probe trials (n = 10)
raw_fea = features # raw data of features (in dictionary form)
theta = 1 # threshold
probability = 0.2 # range = (0, 1)
noise = 0 # delete spikes --> 0, add spikes --> 1
jitter_mean = 0 # Gaussian distribution
jitter_SD = 2 # Gaussian distribution (in number of bins)

R_fea_list, R_null_list = noisy_neural_response(omega, l_curve_inputs, raw_fea, T_fea, theta, probability, noise, jitter_mean, jitter_SD) 
#np.savez("noisy_theta_neural_responses_5F5S", R_fea_list, R_null_list)
np.savez("noisy_corr_neural_responses_5F5S", R_fea_list, R_null_list)
